# Facebook Project

The propose is getting information of pages and write it into files.  Page, posts, comment and reaction of user are inculded. 

Table of Contents.
* [Import Packages.](#Import-Packages.)
* [Setting Token and Authentication.](#Setting-Token-and-Authentication.)
* [Getting Current Date and Time.](#Getting-Current-Date-and-Time.)
* [Create Getting Reaction Information's Function.](#Create-Getting-Reaction-Information's-Function.)
* [Create Getting Tag's Information in Comment Function.](#Create-Getting-Tag's-Information-in-Comment-Function.)
* [Create Getting Comment Information's Function.](#Create-Getting-Comment-Information's-Function.)
* [Create Getting Post Information's Function.](#Create-Getting-Post-Information's-Function.)
* [Create Getting Page Information's Function.](#Create-Getting-Page-Information's-Function.)

## Import Packages.

Import following packages to get the propose.
    1. facepy - The package that help to get information of facebook.
    2. JSON - The data of facebook is JSON format. We need this to work with them.
    3. datatime - Just get current date and time to manage data later.

In [1]:
# Import packages.
import json 
import facepy
import datetime

## Setting Token and Authentication.

Set the token into "token" variable and call "facepy.GraphAPI()" function to authen the token.

If you haven't has token yet, you need to go to [facebook for developer link](https://developers.facebook.com) and sign up first. After you signed up, go to [Explorer](https://developers.facebook.com/tools/explorer/) to get your token.

In [2]:
#Set token.
token = 'Put Your Token'

#To Authen your token to facebook. Version 2.10 is the last updated version since oct, 2017. 
graph = facepy.GraphAPI(token, version='2.10')

## Getting Current Date and Time.

Current date and time will stamp on filename and data to be as of date.  

In [3]:
#Current Datetime
now = datetime.datetime.now()

## Create Getting Reaction Information's Function. 

This function call to facebook to get reaction information.

Input:
    1. Object ID - The Object's ID that we need to get reaction's information.
    2. As of Date - This date will stamp into the data and a part of filename.
    3. Object Type - Type of Object, choose between 'post' and 'comment'.
    
Output:

    This function will create "JSON" file format with "UTF-8" encoding. They will put on Reaction's path on Comment_Reaction's path under the Root's path of project.
    

In [4]:
#Function get_reaction_info, get all reaction on specific object.
def get_reaction_info(object_id, as_of_date, object_type):
    #Set URL
    url = object_id +'/reactions'
    filename = ''
    
    #Setting variable depend on object's type.
    if object_type == 'post':
        object_tag_name = 'post_id'
        object_prefix_filename = 'Reaction/REACTION_'
    elif object_type == 'comment':
        object_tag_name = 'comment_id'
        object_prefix_filename = 'Comment_Reaction/COMMENT_REACTION_'  
    else:
        #Object_type's value don't match. put error message. 
        print('Error type.')
        return
        
    #Get reaction's information.
    reaction_info = graph.get(url, limit='1000')
    
    #Set variables.
    v_check = True
    file_seq = 1
    
    #Loop all data paging to get all reaction. If it don't have Next's tag under Paging's tag, No next's page of data. 
    while(v_check is not False):
        try:
            next_msg = reaction_info['paging']['next']
        except KeyError:
            v_check = False
        
        if len(reaction_info['data']) != 0:
            del reaction_info['headers']
            del reaction_info['paging']
            reaction_info[object_tag_name] = object_id
            reaction_info['as_of_date'] = as_of_date.isoformat()
            
            
            #Set filename
            filename = object_prefix_filename + object_id + '_' + str(file_seq) + '_' + as_of_date.strftime('%Y%m%d') +'.JSON'
            #print(filename)
            #Write output's files.
            with open(filename, 'w', encoding='utf8') as reaction_output_file:
                json.dump(reaction_info, reaction_output_file, ensure_ascii = False)
       
        if v_check is not False:
            next_msg = next_msg[32:]
            reaction_info = graph.get(next_msg)
            file_seq = file_seq + 1
        

In [ ]:
############################################################
#Testing get_reaction_info() function.
############################################################

t_object_id = 'Put Your Oject ID'
t_object_type = 'Put post or comment'
get_reaction_info(t_post_id, now, t_object_type)

#Let's check the files.

## Create Getting Tag's Information in Comment Function. 

This function call to facebook to get tag's information in comments.

Input:
    1. Comment ID - The comment's ID that we need to get tag's information.
    2. As of Date - This date will stamp into the data and a part of filename.
    
Output:

    This function will create "JSON" file format with "UTF-8" encoding. They will put on Comment_Tag's path under the Root's path of project.
    

In [5]:
def get_comment_tag_info(comment_id, as_of_date):
    #Set URL.
    url = comment_id + '?fields=id,message_tags{id,name,type}'
    
    #Get tag's information.
    tag_info = graph.get(url)
    
    #Set variables.
    v_check = True
    
    #Check message_tags's tag.
    try:
        tag = tag_info['message_tags']
        if len(tag) == 0:
            v_check = False
            #print('Tag == 0')
    except KeyError:
        v_check = False
        #print('Do not have message_tags.')
    
    if v_check is not False:
        #Delete and add tags.
        del tag_info['headers']
        tag_info['as_of_date'] = as_of_date.isoformat()
    
        #Set filename
        filename = 'Comment_Tag/COMMENT_TAG_' + comment_id + '_' + as_of_date.strftime('%Y%m%d') +'.JSON'
        
        #Write output's files.
        with open(filename, 'w', encoding='utf8') as tag_output_file:
            json.dump(tag_info, tag_output_file, ensure_ascii = False)
       

In [ ]:
############################################################
#Testing get_comment_tag_info() function.
############################################################

t_comment_id = 'Put Your Comment ID'
get_comment_tag_info(t_comment_id, now)

#Let's check the files.

## Create Getting Comment Information's Function. 

This function call to facebook to get comment's information.

Input:
    1. Post ID - The post's ID that we need to get comment's information.
    2. As of Date - This date will stamp into the data and a part of filename.
    
Output:

    This function will create "JSON" file format with "UTF-8" encoding. They will put on Comment's path under the Root's path of project.
    

In [6]:
#Function get_comment_info, get all comment on specific post.
def get_comment_info(post_id, as_of_date):
    #Set URL.
    url = post_id +'/comments?fields=created_time, id, from, message, like_count'
    filename = ''
    #Get comment's information.
    comment_info = graph.get(url)
    
    #Set variables.
    v_check = True
    file_seq = 1
    
    #Loop all data paging to get all posts. If it don't have Next's tag under Paging's tag, No next's page of data. 
    while(v_check is not False):
        try:
            next_msg = comment_info['paging']['next']
        except KeyError:
            v_check = False
        
        if len(comment_info['data']) != 0:
            del comment_info['headers']
            del comment_info['paging']
            comment_info['post_id'] = post_id
            comment_info['as_of_date'] = as_of_date.isoformat()
            
            #Set filename
            filename = 'Comment/COMMENT_' + post_id + '_' + str(file_seq) + '_' + as_of_date.strftime('%Y%m%d') +'.JSON'
            
            #Write output's files.
            with open(filename, 'w', encoding='utf8') as comment_output_file:
                json.dump(comment_info, comment_output_file, ensure_ascii = False)
           
            #Loop to get any tag and reaction on comments.
            comment_list = comment_info['data']
            for comment in comment_list:
                get_reaction_info(comment['id'], as_of_date, 'comment')
                get_comment_tag_info(comment['id'], as_of_date)
       
        if v_check is not False:
            next_msg = next_msg[32:]
            comment_info = graph.get(next_msg)
            file_seq = file_seq + 1
        


In [9]:
############################################################
#Testing get_comment_info() function.
############################################################

t_post_id = 'Put Your Post ID'
get_comment_info(t_post_id, now)

#Let's check the files.

## Create Getting Post Information's Function. 

This function call to facebook to get post's information.

Input:
    1. Page ID - The page's ID that we need to get post's information.
    2. As of Date - This date will stamp into the data and a part of filename.
    3. Since Date - This function will get posts that they were created after Since's Date.
    4. Until Date - This function will get posts that they were created before Until's Date.
    
Output:

    This function will create "JSON" file format with "UTF-8" encoding. They will put on Post's path under the Root's path of project.
    
Note: This function will automatic call to "get_comment_info()", "get_comment_tag_info()" and "get_reaction_info()" function with Post's ID that they were got.
    

In [7]:
#Function get_post_info, get post of the page on specific time period.
def get_post_info(page_id, as_of_date, since_date, until_date):
    
    #Set URL.
    url = page_id +'/posts?fields=id,name,message,created_time,description,likes.limit(0).summary(total_count),shares,link,type'
    
    #Get post's information.
    post_info = graph.get(url, since=since_date, until=until_date)
    
    #Set variables.
    v_check = True
    file_seq = 1
    
    #Loop all data paging to get all posts. If it don't have Next's tag under Paging's tag, No next's page of data. 
    while(v_check is not False):
        try:
            next_msg = post_info['paging']['next']
        except KeyError:
            v_check = False
        
        if len(post_info['data']) != 0:
            #Delete and add tags.
            del post_info['headers']
            del post_info['paging']
            post_info['page_id'] = page_id
            post_info['as_of_date'] = as_of_date.isoformat()
        
            #Set filename
            filename = 'Post/POST_' + page_id + '_' + str(file_seq) + '_' + as_of_date.strftime('%Y%m%d') +'.JSON'
            
            #Write output's files.
            with open(filename, 'w', encoding='utf8') as post_output_file:
                json.dump(post_info, post_output_file, ensure_ascii = False)
            
            #Call "get_comment_info()" and "get_reaction_info()" functions.
            post_list = post_info['data']
            for post in post_list:
                get_comment_info(post['id'], as_of_date)
                get_reaction_info(post['id'], as_of_date, 'post')
            
        if v_check is not False:
            next_msg = next_msg[32:]
            post_info = graph.get(next_msg)
            file_seq = file_seq + 1
     

In [ ]:
############################################################
#Testing get_post_info() function.
############################################################

t_page_id = 'Put Your Page ID'

#Format of Date should be 'YYYY-MM-DD'
since_date = 'Put Your Since Date'
until_date = 'Put Your Until Date'

get_post_info(t_page_id, now, since_date, until_date)

#Let's check the files.

## Create Getting Page Information's Function. 

This function call to facebook to get page's information.

Input:
    1. Page ID - The page's ID that we need to get page's information.
    2. As of Date - This date will stamp into the data and a part of filename.
    
Output:

    This function will create "JSON" file format with "UTF-8" encoding. They will put on Page's path under the Root's path of project.
    

In [8]:
#Function get_page_info, get page information and write it into a file(Page's type).
def get_page_info(page_id, as_of_date):
    
    #Set URL and Filename.
    url = page_id + '?fields=id,name,fan_count'
    filename = 'Page/PAGE_'  + page_id + '_' + as_of_date.strftime('%Y%m%d') +'.JSON'
    
    #Get  Information from Facebook
    page_info = graph.get(url)
    
    #Delete headers's tag and add as_of_date's tag.
    del page_info['headers']
    page_info['as_of_date'] = as_of_date.isoformat()
    
    #Write output's file.
    with open(filename, 'w', encoding='utf8') as file_output:
        json.dump(page_info, file_output, ensure_ascii = False)
    

In [9]:
############################################################
#Testing get_page_info() function.
############################################################

t_page_id = 'Put Your Page ID'
get_page_info(t_page_id,  now)

#Let's check the files.